In [2]:
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image

def plot_and_save_recalls(file_path, recall_columns, rolling_window=5, output_dir='plots'):
    # Create output directory if it doesn't exist
    import os
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # Read the CSV file
    data = pd.read_csv(file_path)
    
    # Clean the column names by stripping leading/trailing spaces
    data.columns = data.columns.str.strip()
    
    # Extract the epoch column
    epochs = data['epoch']
    
    # Create subplots
    num_plots = len(recall_columns)
    fig, axes = plt.subplots(num_plots, 1, figsize=(10, 6 * num_plots))
    
    # Ensure axes is iterable if there's only one plot
    if num_plots == 1:
        axes = [axes]
    
    image_paths = []
    
    for i, recall_column in enumerate(recall_columns):
        # Extract the recall column
        recall = data[recall_column]
        
        # Calculate the rolling average for smoothing
        recall_smooth = recall.rolling(window=rolling_window).mean()
        
        # Plot the recall and smoothed recall
        axes[i].plot(epochs, recall, marker='o', linestyle='-', color='b', label=recall_column)
        axes[i].plot(epochs, recall_smooth, linestyle='--', color='orange', label=f'{recall_column} (smooth)')
        axes[i].set_title(f'Recall: {recall_column}')
        axes[i].set_xlabel('Epoch')
        axes[i].set_ylabel('Recall')
        axes[i].legend()
        axes[i].grid(True)
        
        # Save individual plot as JPEG
        plot_path = os.path.join(output_dir, f'{recall_column.replace("/", "_")}.jpg')
        fig_single, ax_single = plt.subplots()
        ax_single.plot(epochs, recall, marker='o', linestyle='-', color='b', label=recall_column)
        ax_single.plot(epochs, recall_smooth, linestyle='--', color='orange', label=f'{recall_column} (smooth)')
        ax_single.set_title(recall_column)
        ax_single.set_xlabel('Epoch')
        ax_single.legend()
        ax_single.grid(True)
        fig_single.savefig(plot_path)
        plt.close(fig_single)
        image_paths.append(plot_path)
    
    # Adjust layout and save the composite image
    plt.tight_layout()
    composite_path = os.path.join(output_dir, 'composite.jpg')
    fig.savefig(composite_path)
    plt.close(fig)
    
    # Create a composite image of all individual plots
    images = [Image.open(path) for path in image_paths]
    widths, heights = zip(*(img.size for img in images))
    
    max_width = max(widths)
    total_height = sum(heights)
    
    composite_image = Image.new('RGB', (max_width, total_height))
    
    y_offset = 0
    for img in images:
        composite_image.paste(img, (0, y_offset))
        y_offset += img.height
    
    composite_image.save(composite_path)
    print(f"Composite image saved at {composite_path}")

# Example usage
file_path = 'C:\\repos\\python\\Bacteria_counter\\validation_testing_etc\\validation_USINGMATPLOT\\960px-60_epoch-yolom-augment-updated-10patience.csv'
recall_columns = [
    'train/box_loss', 'train/cls_loss', 'train/dfl_loss', 'metrics/precision(B)',
    'metrics/recall(B)', 'metrics/mAP50(B)', 'metrics/mAP50-95(B)', 'val/box_loss',
    'val/cls_loss', 'val/dfl_loss', 'lr/pg0', 'lr/pg1', 'lr/pg2'
]
plot_and_save_recalls(file_path, recall_columns, rolling_window=5)


Composite image saved at plots\composite.jpg
